In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import dask
import holoviews as hv
import hvplot.xarray
import cftime

import ismip6_helper

In [ ]:
hv.extension('bokeh')

In [ ]:
ismip6_df = ismip6_helper.get_file_index(cache_path="../cache/ismip6_index.parquet")
ismip6_df = ismip6_df.query('variable in ["base"] and institution in ["AWI"]')

In [ ]:
row = ismip6_df.iloc[0]

print(row)

ds = ismip6_helper.open_ismip6_dataset(row['url'])
ds = ismip6_helper.normalize_time_encoding(ds)
ds = ismip6_helper.correct_grid_coordinates(ds, row['variable'])
ds

In [ ]:
# Open the same file with h5netcdf driver to compare
import h5netcdf
import fsspec

f = fsspec.open(row['url']).open()
ncfile = h5netcdf.File(f, 'r')
ncfile

In [ ]:
ncfile.variables['base']

In [ ]:
ds.lithk.hvplot()

In [ ]:
ds.base.hvplot(x='x', y='y').opts(aspect='equal')

In [ ]:
ds.lithk.isel(time=0)[:5, :5].compute()

In [ ]:
ds.lithk.isel(time=0).plot()

In [ ]:


# vds_var_fixed_time = ismip6_helper.fix_time_encoding(vds_var)
# vds_var_normalized_time = ismip6_helper.normalize_time_encoding(vds_var_fixed_time)
# vds_var_fixed_grid = ismip6_helper.correct_grid_coordinates(vds_var_normalized_time, _parse_variable_from_url(url))
        

In [ ]:
# Create a dataframe index by scanning the ISMIP6 Antarctica output files
# This is ~200 lines of code to build an index of ISMIP6 data files from the filenames
ismip6_df = ismip6_helper.get_file_index(cache_path="../cache/ismip6_index.parquet")

# For the purposes of this demo, filter down the number of files we have to load
ismip6_df = ismip6_df.query('experiment in ["ctrl_proj_std", "exp05", "ctrl_proj"] and variable in ["lithk", "base", "sftgrf"] and institution in ["JPL1", "AWI", "DOE"]')

# Build a DataTree of the outputs
datasets = {}
for _, row in ismip6_df.iterrows():
    try:
        p = f'{row["institution"]}_{row["model_name"]}/{row["experiment"]}/{row["variable"]}' # DataTree path
        
        # Use the new helper function that automatically fixes time encoding issues
        ds = ismip6_helper.open_ismip6_dataset(row["url"], chunks={'time': 1})
        ds = ismip6_helper.correct_grid_coordinates(ds, data_var=row["variable"])

        datasets[p] = ds
    except Exception as e:
        print(f"Failed to load {p}: {e}")
        
ismip6_dt = xr.DataTree.from_dict(datasets)

#ismip6_dt

### Select and plot one variable

Once we have the DataTree loaded, we can easily filter down to variables of interest: `ismip6_dt['JPL1_ISSM']['exp05']['lithk']`

**This part works well enough.**

The example below produces a plot of the change in ie thickness since the beginning of the simulation. So far, we've only lazily loaded the data, so the actual data hasn't been downloaded. We call `.compute()` on the thickness change variable to force loading of the data in order to make the interactive plot responsive.

In [ ]:
# Select one dataset from the DataTree
dt = ismip6_dt['JPL1_ISSM']['exp05']['lithk']

# Compute the change in thickness relative to the first time step
# Since the datasets are lazily loaded, we now want to actually force computation of a result
# so that the interactive plot will be responsive.
delta_thickness = (dt['lithk'] - dt['lithk'].isel(time=0)).rename('delta_lithk').compute()

# Determine a useful color scale range
vmag = np.max(np.abs(delta_thickness.quantile([0.01, 0.99]).values))

# Plot with a slider to change the date
delta_thickness.hvplot.image(x='x', y='y', clim=(-vmag, vmag), cmap='RdBu').opts(
        aspect='equal',
        title="Change in ice thickness relative to the first timestep",
        colorbar_opts={'title': 'Change in thickness (m)'},
    )

### Regridding multiple models to a common comparison grid

While all of the ISMIP6 outputs were interpolated to a regular grid, these grids have different resolutions. So if we want to do any cross-model comparison, we need to get things onto a common grid.

We have some more complicated ideas about how to do regridding, but we also want to make sure that simple things work.

Ideally, it would be possible to call `interp` on a DataTree like this:

```python
comparison_grid = xr.Dataset({
    'x': (['x'], np.arange(-3040e3, 3040e3, 16e3)),
    'y': (['y'], np.arange(-3040e3, 3040e3, 16e3)),
    'time': (['time'], xr.date_range('2016-01-01', '2100-12-31', freq='10YS').values),
})

ismip6_dt_regridded = ismip6_dt.interp(x=comparison_grid.x, y=comparison_grid.y)
```

This doesn't actually work yet, but we can use `map_over_datasets` to do the same thing. Not terrible, but could be cleaner.

The other issue is that the CMIP standard (inherited by ISMIP) is one variable per file. This leads to the following structure:

```
model
 |-> var1
 |-> var2
model2
 |-> var1
 |-> var2
```

This makes it hard to manipulate variables together because map_over_datasets doesn’t easily give access to children of the nodes. So instead we manually condense the leaves of the tree.

In [ ]:
comparison_grid = xr.Dataset({
    'x': (['x'], np.arange(-3040e3, 3040e3, 16e3)),
    'y': (['y'], np.arange(-3040e3, 3040e3, 16e3)),
    'time': (['time'], xr.date_range('2016-01-01', '2100-12-31', freq='10YS').values),
})

regridded = ismip6_dt.map_over_datasets(
    lambda x: x.interp(
        x=comparison_grid.x,
        y=comparison_grid.y,
        time=comparison_grid.time,
        method='linear'
    ) if ('x' in x.dims and 'y' in x.dims and 'time' in x.dims) else x
)

def condense_datatree(dt):
    dt = dt.copy()
    if dt.is_hollow and all(child.is_leaf for child in dt.children.values()):
        # TODO: Should actually check that the dimensions are compatible
        return xr.merge([child.ds for child in dt.children.values()])
    for child_name, child in dt.children.items():
        dt[child_name] = condense_datatree(child)
    return dt

regridded = condense_datatree(regridded)
ismip6_dt_condensed = condense_datatree(ismip6_dt) # Also condense the non-regridded for later

Now that we're working on a common comparison grid, we can do some cross-model comparison. As an example, we'll plot the standard deviation of the change in ice thickness since the first timestep of each model.

This works but it's a bit ugly, mostly because our list comprehension has to test if we're on the right node level. Possibly a better solution might look like:

```python
ismip6_dt.match['*/exp05']['lithk'].std()
```

Or just

```python
xr.concat(regridded.match("*/*"), dim='model').std(dim='model')
```

In [ ]:
# Calculate the standard deviation of the change in lithk across models
delta_lithk_all = xr.concat([
    (node.ds['lithk'].isel(time=slice(1, None)) - node.ds['lithk'].isel(time=0)) 
    for node in regridded.subtree 
    if node.path.endswith('exp05') and node.has_data
], dim='model').std(dim='model').compute()

delta_lithk_all.hvplot.image(
    x='x', y='y', 
    clim=(0, 200), 
    cmap='gray_r',
    clabel='Std dev of thickness change (m)'
).opts(aspect='equal', title='Standard deviation of ice thickness change across models')

### Computed scalars: mass above flotation

As a further example, we will re-create a very small part of the analysis done for this paper:

> Seroussi, H., Nowicki, S., Payne, A. J., Goelzer, H., Lipscomb, W. H., Abe-Ouchi, A., Agosta, C., Albrecht, T., Asay-Davis, X., Barthel, A., Calov, R., Cullather, R., Dumas, C., Galton-Fenzi, B. K., Gladstone, R., Golledge, N. R., Gregory, J. M., Greve, R., Hattermann, T., Hoffman, M. J., Humbert, A., Huybrechts, P., Jourdain, N. C., Kleiner, T., Larour, E., Leguy, G. R., Lowry, D. P., Little, C. M., Morlighem, M., Pattyn, F., Pelle, T., Price, S. F., Quiquet, A., Reese, R., Schlegel, N.-J., Shepherd, A., Simon, E., Smith, R. S., Straneo, F., Sun, S., Trusel, L. D., Van Breedam, J., van de Wal, R. S. W., Winkelmann, R., Zhao, C., Zhang, T., and Zwinger, T.: ISMIP6 Antarctica: a multi-model ensemble of the Antarctic ice sheet evolution over the 21st century, The Cryosphere, 14, 3033–3070, https://doi.org/10.5194/tc-14-3033-2020, 2020.

(With results archived at https://zenodo.org/records/3940766)

Specifically, we will calculate the summed volume above flotation for one model. In the cell below, we load the NetCDF files provided at the Zenodo archive above and plot them for reference.

In [ ]:
tmp_ivaf_reference = xr.open_dataset('external_data/ismip6_computed_scalars/computed_ivaf_AIS_JPL1_ISSM_exp05.nc', engine='h5netcdf', decode_times=False)
tmp_ivaf_minus_ctrl_reference = xr.open_dataset('external_data/ismip6_computed_scalars/computed_ivaf_minus_ctrl_proj_AIS_JPL1_ISSM_exp05.nc', engine='h5netcdf', decode_times=False)

ivaf_reference = xr.merge([tmp_ivaf_reference['ivaf'], tmp_ivaf_minus_ctrl_reference['ivaf'].rename('ivaf_minus_ctrl')])
(ivaf_reference - ivaf_reference.isel(time=0)).hvplot.line(
    x='time',
    ylabel='Cumulative change in volume above flotation (m^3)',
    title='Ice Volume Above Flotation for JPL1_ISSM exp05'
    )

We need a few approximate constants (below) and then we also need to know the areal scaling factor of the EPSG:3031 projection, which is only true scale at -71 degrees south. To verify the result, we first compute it using PyProj and then validate it against the NetCDF file provided in the Zenodo link.

In [ ]:
ice_density = 917 # kg/m^3
ocean_density = 1028 # kg/m^3

In [ ]:
# Calculate the scaling factor for each grid cell in regridded using pyproj
# The scaling factor accounts for map projection distortion
import pyproj

# Create 2D grids for x and y
xx, yy = np.meshgrid(comparison_grid.x.values, comparison_grid.y.values)

# Set up the projection for EPSG:3031 (Antarctic Polar Stereographic)
proj = pyproj.Proj('EPSG:3031')

# Convert projected coordinates to lat/lon
lons, lats = proj(xx, yy, inverse=True)

# Get the factors at each grid point using the Proj object
# get_factors returns: (meridional_scale, parallel_scale, areal_scale, 
#                       angular_distortion, meridian_parallel_angle, 
#                       meridian_convergence, tissot_semimajor, tissot_semiminor)
# We want the areal_scale (index 2)
factors = proj.get_factors(lons.ravel(), lats.ravel(), radians=False)
areal_scale = factors.areal_scale.reshape(xx.shape)

# Create the scaling factor as an xarray DataArray
scale_factor = xr.DataArray(
    areal_scale,
    coords={'y': comparison_grid.y.values, 'x': comparison_grid.x.values},
    dims=['y', 'x'],
    name='scalefac',
    attrs={
        'long_name': 'Area scaling factor',
        'description': 'Ratio of true area to projected area from pyproj.get_factors',
        'units': '1',
        'projection': 'EPSG:3031',
    }
)

scale_factor.hvplot.image().opts(aspect='equal')

In [ ]:
scale_file = xr.open_dataset('external_data/ismip6_computed_scalars/af2_el_ismip6_ant_01.nc')
scale_file_interp = scale_file['af2'].interp(x=comparison_grid.x, y=comparison_grid.y, method='linear')

(scale_file_interp - (1/scale_factor)).hvplot.image().opts(aspect='equal', clim=(-0.1, 0.1), title='Difference between computed and reference scaling factor')

Now we can define a function to find the volume above flotation and apply it to every model in our DataTree.

The main syntax awkwardness here is having to check if we're on the right node level within the helper function:

```python
# Check if we're on a node that has the required variables
if 'lithk' not in dt or 'base' not in dt or 'sftgrf' not in dt:
    return dt
```

In [ ]:
def calc_ivaf(dt):
    # Check if we're on a node that has the required variables
    if 'lithk' not in dt or 'base' not in dt or 'sftgrf' not in dt:
        return dt
    
    lithk = dt['lithk']
    base = dt['base']
    sftgrf = dt['sftgrf']

    resolution = dt['x'][1] - dt['x'][0]  # in meters

    ivaf = ((lithk + ocean_density/ice_density * np.minimum(base, 0)) * sftgrf * (1/scale_factor)).sum(dim=['x', 'y']) * (resolution**2)  # in m^3
    dt = dt.copy()
    dt['ivaf'] = ivaf.rename('ivaf')
    return dt

regridded = regridded.map_over_datasets(calc_ivaf)

We've now setup the calculation for `ivaf` across every model, but not actually run it. For the plots below, we'll force computation of the volume above flotation for `exp05` and the same with the control run subtracted.

In [ ]:
ivaf = regridded['JPL1_ISSM']['exp05']['ivaf'].compute()
ivaf_minus_ctrl = (ivaf - regridded['JPL1_ISSM']['ctrl_proj']['ivaf']).compute()

In [ ]:
calc = ((ivaf - ivaf.isel(time=0)).hvplot.line(x='time', label='exp05') *
        (ivaf_minus_ctrl - ivaf_minus_ctrl.isel(time=0)).hvplot.line(x='time', label='exp05 - ctrl_proj')
        ).opts(
    ylabel='Cumulative change in\nvolume above flotation (m^3)',
    title='Computed changed in ice volume above flotation for JPL1_ISSM exp05',
    legend_position='bottom_left', show_grid=True
    )
ref = (ivaf_reference - ivaf_reference.isel(time=0)).hvplot.line(
    x='time',
    ylabel='Cumulative change in\nvolume above flotation (m^3)',
    title='Reference (from doi.org/10.5281/zenodo.3940766)'
    ).opts(legend_position='bottom_left', show_grid=True)

calc + ref

On the left are the values we computed here, compared with the reference values on the right from the Zenodo archive. We computed ours on a resampled grid, so it's not an exact match.